In [1]:
train_file="/home/harshit/Desktop/engtocmd/data/nkjcb_data.csv"
word_vec_file="/home/harshit/Desktop/engtocmd/data/wiki-news-300d-1M.vec"
import pandas as pd
import numpy as np
import tensorflow
from tensorflow import keras
from tensorflow.keras.layers import *

In [2]:
df=pd.read_csv(train_file)
print(df.shape)
print(df.iloc[10])

(34, 4)
id                            10
tokens    go to abc123 directory
slots              o o dirname o
intent                        cd
Name: 10, dtype: object


In [3]:
df

,id,tokens,slots,intent
0,0,create a directory on the desktop,o o o o o dirloc,mkdir
1,1,create a directory,o o o,mkdir
2,2,create a folder,o o o,mkdir
3,3,create a folder named abc at dekstop/tp,o o o o dirname o dirloc,mkdir
4,4,create a folder named xyz at dekstop,o o o o dirname o dirloc,mkdir
5,5,create a folder named xyz,o o o o dirname,mkdir
6,6,create a folder xyz,o o o dirname,mkdir
7,7,create a directory xyz,o o o dirname,mkdir
8,8,i want a folder named xyz123 at location deskt...,o o o o o dirname o o dirloc,mkdir
9,9,move to abc123 directory,o o dirname o,cd


In [4]:
n=df.to_numpy()
intents=[]
slots=[]
for row in n:
    for i,val in enumerate(row):
        if i==3:
            if val not in intents:
                intents.append(val)
        elif i==2:
            for slot in val.split():
                if slot not in slots:
                    slots.append(slot)
print(len(intents))
print(intents)
print(len(slots))
print(slots)

5
['mkdir', 'cd', 'ls', 'touch', 'man']
5
['o', 'dirloc', 'dirname', 'filname', 'comname']


In [5]:
import numpy as np
import io


def vocab(embf=None, n=100000, req=set()):
    if not embf:
        embf=word_vec_file
    def load_vectors(fname, n):
        fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
        # n, d = map(int, fin.readline().split())
        data = {}
        for i, line in enumerate(fin):
            if i == 0:
                continue            
            tokens = line.rstrip().split(' ')            
            if i<n:
                data[tokens[0]] = list(map(float, tokens[1:]))
            elif tokens[0] in req:
                data[tokens[0]] = list(map(float, tokens[1:]))
        return data

    word_to_vec = {}
    unk = [0] *300
    word_to_vec = load_vectors(embf, n)    
    return word_to_vec

In [6]:
def gen(train_set, intents, slots, word_to_vec):
    while True:        
        for row in train_set:            
            tokens=row[1].split()
            ip=np.zeros((1,len(tokens),300)) 
            for i, token in enumerate(tokens):
                if token in word_to_vec:
                    ip[0,i]=word_to_vec[token]
                else:
                    ip[0,i]=[0]*300
            op_intent=np.zeros((1, len(intents)))
            op_intent[0, intents.index(row[3])]=1
            op_slots=np.zeros((1,len(tokens),len(slots)))
            r_slots=row[2].split()
            if not len(r_slots) == len(tokens):
              print(tokens)
              print(r_slots)
              continue
            for i,slot in enumerate(r_slots):
                op_slots[0,i,slots.index(slot)]=1
                
            yield ip, (op_intent, op_slots)

In [7]:
word_to_vec=vocab(n=1*10**5)
g=gen(n, intents, slots, word_to_vec)
ip, op=g.__next__()
print(op[0])
print(op[1])

[[1. 0. 0. 0. 0.]]
[[[1. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]]]


In [8]:
n=df.to_numpy()
ip=Input(name="Input",batch_shape=(1,None,300))
enc_seq, enc_final_state=GRU(units=100, name="Encoder", return_sequences=True, return_state=True)(ip)
intent_classifier=Dense(units=len(intents), activation="softmax", name="intent_classifier")(enc_final_state)

decoder_seq=GRU(units=100 ,return_sequences=True, name="Decoder")(enc_seq, initial_state=enc_final_state)
slots_tagger=Dense(units=len(slots), activation="softmax", name="slots_tagger")(decoder_seq)
model=keras.models.Model(inputs=ip,outputs=(intent_classifier, slots_tagger),name="intent_and_slots")
model.summary()
# input()

# word_to_vec=vocab(n=1*10**5)
model.compile(optimizer=keras.optimizers.Adam(), loss="categorical_crossentropy", loss_weights=[1.0, 1.0])
model.fit_generator(gen(n, intents, slots, word_to_vec), epochs=15, steps_per_epoch=n.shape[0])

W1005 22:09:04.164867 139728264197952 deprecation.py:506] From /home/harshit/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "intent_and_slots"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(1, None, 300)]     0                                            
__________________________________________________________________________________________________
Encoder (GRU)                   [(1, None, 100), (No 120300      Input[0][0]                      
__________________________________________________________________________________________________
Decoder (GRU)                   (1, None, 100)       60300       Encoder[0][0]                    
                                                                 Encoder[0][1]                    
__________________________________________________________________________________________________
intent_classifier (Dense)       (None, 5)            505         Encoder[0][1]     

W1005 22:09:07.043767 139728264197952 deprecation.py:323] From /home/harshit/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/15
34/34 [==============================] - 4s 123ms/step - loss: 2.8479 - intent_classifier_loss: 1.8822 - slots_tagger_loss: 0.9657
Epoch 2/15
34/34 [==============================] - 1s 15ms/step - loss: 1.9560 - intent_classifier_loss: 1.3464 - slots_tagger_loss: 0.6097
Epoch 3/15
34/34 [==============================] - 1s 15ms/step - loss: 1.5030 - intent_classifier_loss: 1.0609 - slots_tagger_loss: 0.4422
Epoch 4/15
34/34 [==============================] - 1s 15ms/step - loss: 1.0177 - intent_classifier_loss: 0.7284 - slots_tagger_loss: 0.2893
Epoch 5/15
34/34 [==============================] - 1s 15ms/step - loss: 0.7935 - intent_classifier_loss: 0.5703 - slots_tagger_loss: 0.2232
Epoch 6/15
34/34 [==============================] - 1s 15ms/step - loss: 0.5220 - intent_classifier_loss: 0.3675 - slots_tagger_loss: 0.1545
Epoch 7/15
34/34 [==============================] - 1s 16ms/step - loss: 0.3430 - intent_classifier_loss: 0.2500 - slots_tagger_loss: 0.0930
Epoch 8/15
3

In [ ]:
while True:
    sent=input("Enter\n")
    tokens=sent.split()
    ip=np.zeros((1,len(tokens),300)) 
    for i, token in enumerate(tokens):
        if token in word_to_vec:
            ip[0,i]=word_to_vec[token]
        else:
            ip[0,i]=[0]*300
    pred=model.predict(ip)
    intents_pred=pred[0]
    intents_pred=list(np.squeeze(intents_pred))
    print(intents_pred)
    print("Intent:", intents[intents_pred.index(max(intents_pred))])
    slots_pred=pred[1]
    slots_pred=np.squeeze(slots_pred)
    for pred in slots_pred:
        pred=list(pred)        
        print(slots[pred.index(max(pred))], end=" ")

In [9]:
model.save("kj_cb-e17-nd100k.h5")